# Data-Parallel Programming


Central idea in task parallelism is to express parts of computation that should proceed in parallel. Here is definition is task parallelism

> A form of parallelization that distributes execution processes across computing nodes.

We know how to express parallel programs with task and parallel constructs. Data parallelism takes a different approach. Defintion of data parallelism

> A form of parallelization that distributes data across computing nodes.

The simplest form of data-parallel programming is the parallel for loop.

## Example: initializing the array values.

```scala
def initializeArray(xs: Array[Int])(v: Int): Unit = {
for (i <- (0 until xs.length).par) {
xs(i) = v
}
}
```
`intializeArray` is a function that takes an array `xs` and intializes all the entries with value `v` in parallel. The parallel for loop looks like sequential, it calls `.par` on range and returns parallel range. Iterations of for-loop will be executed on different processors concurrently. In the body of for-loop each entry of array `xs` assigned value `v`. Parallel-loop doesn't return anything. Therefore not functional (since sideffects exist). A parallel for-loop is correct only if it writes to seperate memory locations or use synchronized locks. In our example this is assured by modifiying different memory locations.

## Example: Mandelbrot Set

Although simple, parallel for loop allows writing interesting programs.
Render a set of complex numbers in the plane for which the sequence
$z_{n+1} = z^2_n + c$ does not approach infinity (absolute value of numbers do not approach infinity). How do we visualize this on computer image?

To each pixel on the screen, we compute the corresponding point in complex plane, we compute value of $z_n$ upto some maximum value of iterations. If the $|z_i| \geq 2$ then sequence will approach infinity, otherwise if maximum numbers of iteration are reached we assume the sequence will never diverge. The color visually tells us how fast the sequence is diverging.


```scala
private def computePixel(xc: Double, yc: Double, maxIterations: Int): Int = {
var i = 0
var x, y = 0.0
while (x * x + y * y < 4 && i < maxIterations) {
val xt = x * x - y * y + xc
val yt = 2 * x * y + yc
x = xt; y = yt
i += 1
}
color(i)
}
```

`computePixel` takes two co-ordinates `xc` and `yc` and return the color corresponding to number of iterations the loop run.

```scala
def parRender(): Unit = {
for (idx <- (0 until image.length).par) {
val (xc, yc) = coordinatesFor(idx)
image(idx) = computePixel(xc, yc, maxIterations)
}
}
```

Given array of pixels called image, it converts pixel into co-ordinates. It computes the color of pixel and assigns it to image array.

## Workload

Different data-parallel programs have different workloads.

> Workload is a function that maps each input element to the amount of work required to process it.

In the Mandelbrot case: $w(i) = #iterations$

The workload depends on the problem instance.


Goal of the data-parallel scheduler: 

> efficiently balance the workload across processors without any knowledge about the $w(i)$.